In [9]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import dill

import sys
sys.path.append("../../")

import biked_commons
from biked_commons.prediction import aero_predictor, loaders
from biked_commons.resource_utils import models_and_scalers_path
from biked_commons.prediction.prediction_utils import TorchStandardScaler, DNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
X_tv, Y_tv = loaders.load_validity()
X_tv_tens = torch.tensor(X_tv.values, dtype=torch.float32, device=device)
Y_tv_tens = torch.tensor(Y_tv.values, dtype=torch.float32, device=device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [13]:
X_tv

,SSB_Include,CSB_Include,CS Length,BB Drop,Stack,SS E,ST Angle,BB OD,TT OD,HT OD,...,CSB Offset,SS Z,SS Thickness,CS Thickness,TT Thickness,BB Thickness,HT Thickness,ST Thickness,DT Thickness,DT Length
9237,0.0,0.0,0.454210,4.974500e-02,0.56541,0.028226,74.602428,0.03916,0.02972,0.03474,...,0.381567,0.010334,0.006401,0.001903,0.003043,0.003054,0.008850,0.002345,0.006206,0.660599
8922,1.0,1.0,0.428072,7.297700e-02,0.56664,0.034613,72.058553,0.04012,0.02885,0.03471,...,0.299777,0.008548,0.006327,0.002624,0.002506,0.000540,0.002825,0.000872,0.003573,0.674965
11585,1.0,1.0,0.358010,-2.328000e-02,0.55498,0.175450,74.661000,0.04585,0.04080,0.04277,...,0.349621,0.008206,0.003124,0.006949,0.000855,0.005617,0.001092,0.001054,0.004003,0.568519
6892,1.0,1.0,0.404602,7.005700e-02,0.56557,0.044059,74.011709,0.03994,0.02861,0.04496,...,0.350116,0.008996,0.001073,0.001192,0.000751,0.000813,0.000962,0.000798,0.000956,0.658852
4783,1.0,1.0,0.297080,-1.282000e-02,0.56564,0.053940,66.369610,0.04082,0.02912,0.04473,...,0.300662,0.009216,0.001014,0.001998,0.001276,0.000651,0.008124,0.008348,0.005268,0.415527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,0.0,0.0,0.425420,2.526700e-02,0.56560,0.045000,72.488558,0.03904,0.02860,0.04295,...,0.300000,0.009000,0.003461,0.001703,0.003410,0.001102,0.004535,0.002055,0.009791,0.674373
13948,0.0,0.0,0.409670,5.098800e-02,0.56523,0.039210,73.530997,0.04179,0.02866,0.04548,...,0.290824,0.008977,0.002399,0.002827,0.005921,0.002497,0.003042,0.006572,0.002998,0.665984
1307,0.0,0.0,0.362440,1.989520e-15,0.56560,0.045000,74.000000,0.04000,0.02860,0.04500,...,0.300000,0.009000,0.002200,0.002957,0.001814,0.000863,0.001952,0.003190,0.002940,0.570119
8870,1.0,1.0,0.392032,6.675600e-02,0.53600,0.173262,75.240875,0.04003,0.02882,0.04546,...,0.349996,0.008863,0.001042,0.000980,0.003368,0.003429,0.001838,0.003297,0.003799,0.645932


In [5]:
scaler = TorchStandardScaler().to(device)
scaler.fit(X_tv_tens)


scaler_path = models_and_scalers_path("validity_scaler.pt")
torch.save(scaler, scaler_path)

X_tv_tens = torch.tensor(scaler.transform(X_tv_tens), dtype=torch.float32, device=device)

NameError: name 'X_tv_tens' is not defined

In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(X_tv_tens, Y_tv_tens, test_size=0.2, random_state=42)

NameError: name 'X_tv_tens' is not defined

In [ ]:
# Define the model, loss function, and optimizer
input_dim = X_train.shape[1]
output_dim = 1
model = clip_predictor.ResidualNetwork(96, 512, 256, 2, 3)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
batch_size = 32
best_val_loss = float('inf')
model_path = resource_utils.resource_path("models")
save_path = model_path + '/clip.pth'

for epoch in range(num_epochs):
    model.train()
    permutation = torch.randperm(X_train.size()[0])
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, Y_val)

    # Save best model
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        torch.save(model, save_path)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f} (Best: {best_val_loss:.4f})')



Epoch [10/500], Loss: 3.6706, Val Loss: 1.0024 (Best: 1.0024)
Epoch [20/500], Loss: 3.2093, Val Loss: 0.4705 (Best: 0.4547)
Epoch [30/500], Loss: 2.3870, Val Loss: 0.3969 (Best: 0.3969)
Epoch [40/500], Loss: 2.2113, Val Loss: 0.4355 (Best: 0.3321)
Epoch [50/500], Loss: 2.2873, Val Loss: 0.3470 (Best: 0.3103)
Epoch [60/500], Loss: 1.4425, Val Loss: 0.3368 (Best: 0.3103)
Epoch [70/500], Loss: 1.8420, Val Loss: 0.2951 (Best: 0.2885)
Epoch [80/500], Loss: 2.1649, Val Loss: 0.4410 (Best: 0.2885)
Epoch [90/500], Loss: 1.9080, Val Loss: 0.3816 (Best: 0.2721)
Epoch [100/500], Loss: 1.3815, Val Loss: 0.4398 (Best: 0.2721)
Epoch [110/500], Loss: 1.9357, Val Loss: 0.5580 (Best: 0.2721)
Epoch [120/500], Loss: 1.7282, Val Loss: 0.3901 (Best: 0.2721)
Epoch [130/500], Loss: 1.5907, Val Loss: 0.5393 (Best: 0.2545)
Epoch [140/500], Loss: 1.6743, Val Loss: 0.3432 (Best: 0.2545)
Epoch [150/500], Loss: 1.7459, Val Loss: 0.3530 (Best: 0.2545)
Epoch [160/500], Loss: 1.1400, Val Loss: 0.4068 (Best: 0.2535)
E

In [ ]:
model = torch.load(save_path)

C:\Users\Lyler\AppData\Local\Temp\ipykernel_4384\1837872702.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(save_path)


In [ ]:
import evaluators

evaluators.evaluate_aero(model, device=device)

0.9547767043113708

In [ ]:
import pandas as pd
import numpy as np
import dill
model_path = resource_utils.resource_path("models") + '/clip_old.pt'
scaler_path = resource_utils.resource_path("models") + '/clip_old_scaler.pk'

ref_model = ResidualNetwork(96, 512, 256, 2, 3)
ref_model.load_state_dict(torch.load(model_path))
with open(scaler_path, "rb") as file:
    scaler = dill.load(file)
X_test = pd.read_csv('../../resources/datasets/split_datasets/CLIP_X_test.csv', index_col=0)
Y_test = np.load("../../resources/datasets/split_datasets/CLIP_Y_test.npy")

X_test_scaled = scaler.transform(X_test.values)
Y_pred = ref_model(torch.tensor(X_test_scaled, dtype=torch.float32, device=device)).cpu().detach().numpy()


C:\Users\Lyler\AppData\Local\Temp\ipykernel_34848\1547622106.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ref_model.load_state_dict(torch.load(model_path))
c:\Users\L

FileNotFoundError: [Errno 2] No such file or directory: '../../resources/datasets/split_datasets/CLIP_Y_test.npy'

In [ ]:
X_test.columns

Index(['SSB_Include', 'CSB_Include', 'CS Length', 'BB Drop', 'Stack', 'SS E',
       'ST Angle', 'BB OD', 'TT OD', 'HT OD', 'DT OD', 'CS OD', 'SS OD',
       'ST OD', 'CS F', 'HT LX', 'ST UX', 'HT UX', 'HT Angle', 'HT Length',
       'ST Length', 'BB Length', 'Dropout Offset', 'SSB OD', 'CSB OD',
       'Material', 'SSB Offset', 'CSB Offset', 'SS Z', 'SS Thickness',
       'CS Thickness', 'TT Thickness', 'BB Thickness', 'HT Thickness',
       'ST Thickness', 'DT Thickness', 'DT Length'],
      dtype='object')